# Spam Filtering Using The [Enron Dataset][1]
[1]: [http://www.aueb.gr/users/ion/data/enron-spam/]

In [1]:
from pymldb import Connection
mldb = Connection('http://localhost/')

Let's start by loading the dataset. We have already merged the different email files in a sensible manner into a .csv file, which we've made available online. Since this dataset is actually made up of six different datasets, we'll restrict ourself to the first one for simplicity, using a "where" clause.

In [2]:
print mldb.post('/v1/procedures', {
    'type': 'import.text',
    'params': {
        'dataFileUrl': 'http://public.mldb.ai/datasets/enron.csv.gz',
        'outputDataset': 'enron_data',
        'named': "'enron_' + dataset + '_mail_' + index",
        'where': 'dataset = 1',
        'runOnCreation': True
        }
    })

<Response [201]>


This is what the dataset looks like.

*index*: order in which the emails arrived in the user's inbox  
*msg*: actual content of the email  
*label*: was the email legitimate (*ham*) or not (*spam*)  

In [3]:
mldb.query('select index, msg, label from enron_data order by index limit 10')

,index,label,msg
_rowName,,,
enron_1_mail_0,0,spam,Subject: dobmeos with hgh my energy level has ...
enron_1_mail_1,1,spam,Subject: your prescription is ready . . oxwq s...
enron_1_mail_2,2,ham,Subject: christmas tree farm pictures
enron_1_mail_3,3,ham,"Subject: vastar resources , inc .gary , produc..."
enron_1_mail_4,4,ham,Subject: calpine daily gas nomination- calpine...
enron_1_mail_5,5,ham,Subject: re : issuefyi - see note below - alre...
enron_1_mail_6,6,ham,Subject: meter 7268 nov allocationfyi .- - - -...
enron_1_mail_7,7,spam,Subject: get that new car 8434people nowthe we...
enron_1_mail_8,8,ham,"Subject: mcmullen gas for 11 / 99jackie ,since..."


Let's create a *sql.expression* that will simply tokenize the emails into a bag of words. Those will be our features on which we will train a classifier.

In [4]:
print mldb.put('/v1/functions/bow', {
    'type': 'sql.expression',
    'params': {
        'expression': """
            tokenize(msg, {splitchars: ' \n', quotechar: ''}) as bow
            """
    }
})

<Response [201]>


Then we can generate the features for the whole dataset, and write them into a new dataset, using the *transform* procedure.

In [5]:
print mldb.post('/v1/procedures', {
    'type': 'transform',
    'params': {
        'inputData': """
            select bow({msg}) as features, label = 'spam' as label
            from enron_data
            """,
        'outputDataset': 'enron_features',
        'runOnCreation': True
    }
})

<Response [201]>


Here is a snapshot of the sparse feature matrix:

In [6]:
mldb.query('select * from enron_features limit 10')

,features.bow.',features.bow.(,features.bow.)we,"features.bow.,this",features.bow.-,"features.bow."".""","features.bow."".10""",features.bow.01,features.bow.06,features.bow.29,...,features.bow.won,features.bow.work,features.bow.works,features.bow.worldnet,features.bow.woundn,features.bow.xinbox,features.bow.yahoo,features.bow.you>,features.bow.zee,features.bow.zigzag
_rowName,,,,,,,,,,,,,,,,,,,,,
enron_1_mail_3917,1,1,1,1,8,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3904,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_4201,NaN,NaN,NaN,NaN,1,NaN,NaN,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_1388,NaN,NaN,NaN,NaN,47,15,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3765,1,1,NaN,NaN,2,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_4361,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3651,NaN,NaN,NaN,NaN,1,4,NaN,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3040,4,NaN,NaN,NaN,6,10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enron_1_mail_3648,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, let's train a very simple classifier, by training on half of the messages, and testing on the other half. This classifier will give a score to every email, and we can then choose a threshold where everything above the threshold is classified as spam, and every thing below as ham.

In [7]:
n = mldb.get('/v1/query', q='select count(*) as n from enron_features',
             format='aos').json()[0]['n']

res = mldb.post('/v1/procedures', {
    'type': 'classifier.experiment',
    'params': {
        'experimentName': 'enron_experiment1',
        'inputData': 'select {features.*} as features, label from enron_features',
        'datasetFolds': [{
           'trainingWhere': 'rowHash() % 2 = 0',
           'testingWhere': 'rowHash() % 2 = 1'
        }],
        'modelFileUrlPattern': 'file://enron_model_$runid.cls',
        'algorithm': 'bbdt',
        'runOnCreation': True
    }
})

In [8]:
print 'AUC =', res.json()['status']['firstRun']['status']['folds'][0]['resultsTest']['auc']

AUC = 0.993899486647


Not a bad AUC for a model that simple. But [the AUC score of a classifier is only a very generic measure of performance][1]. When having a specific problem like spam filtering, we're better off using a performance metric that truly matches our intuition about what a good spam filter ought to be. Namely, a good spam filtering algorithm should almost never flag as spam a legitime email, while keeping your inbox as spam-free as possible. This is what should be used to choose the threshold for the classifier, and then to measure its performance.

So instead of the AUC (that doesn't pick a specific threshold but uses all of them), let's use as our performance metric the best [$F_{0.05}$ score][2], which gives 20 times more importance to precision than recall. In other words, this metric represents the fact that classifying as spam **only** what is really spam is 20 times more important than finding all the spam.

Let's see how we are doing with that metric.
[1]: http://mldb.ai/blog/posts/2016/01/ml-meets-economics/
[2]: https://en.wikipedia.org/wiki/F1_score

In [9]:
print mldb.put('/v1/functions/enron_score', {
    'type': 'sql.expression',
    'params': {
        'expression': """
            (1 + pow(ratio, 2)) * (precision * recall) / (precision * pow(ratio, 2) + recall) as enron_score
            """
    }
})

<Response [201]>


In [10]:
mldb.query("""
    select "truePositives", "trueNegatives", "falsePositives", "falseNegatives", precision, recall, score,
           enron_score({precision, recall, ratio:0.05}) as *
    from enron_experiment1_results_0
    order by enron_score desc
""")

,enron_score,falseNegatives,falsePositives,precision,recall,score,trueNegatives,truePositives
_rowName,,,,,,,,
enron_1_mail_3120,0.997406,389,0,1.000000,0.489501,2.311491,1881,373
enron_1_mail_2262,0.997392,390,0,1.000000,0.488189,2.322494,1881,372
enron_1_mail_2675,0.997392,390,0,1.000000,0.488189,2.322494,1881,372
enron_1_mail_2942,0.997365,392,0,1.000000,0.485564,2.352247,1881,370
enron_1_mail_2258,0.997351,393,0,1.000000,0.484252,2.360322,1881,369
enron_1_mail_2965,0.997337,394,0,1.000000,0.482940,2.366064,1881,368
enron_1_mail_236,0.997323,395,0,1.000000,0.481627,2.367701,1881,367
enron_1_mail_837,0.997309,396,0,1.000000,0.480315,2.371384,1881,366
enron_1_mail_1659,0.997295,397,0,1.000000,0.479003,2.373372,1881,365


As you can see, the best threshold is one where in case of doubt, almost everything is classified as "ham". On our test set, this leads to 389 spam messages in the inbox, no ham wrongly filtered as spam and 2254 well classified emails. Now how can we improve this?

# To Be Continued...